In [1]:
from ephysvibe.structures.population_data import PopulationData
from ephysvibe.structures.neuron_data import NeuronData
from ephysvibe.trials.spikes import firing_rate
from ephysvibe.trials import select_trials
from ephysvibe.stats import smetrics

import numpy as np

In [ ]:

def get_selectivity_info(neu: NeuronData):
    win = 100
    res = {}
    res["nid"] = neu.get_neuron_id()
    samples = [11, 15, 51, 55, 0]
    inout_n0_fr = []
    inout_nn_fr = []
    for inout in ["in", "out"]:
        mask = getattr(neu, "mask_" + inout)
        sp = getattr(neu, "sample_on_" + inout)
        st = getattr(neu, "st_sample_on_" + inout)
        fr = firing_rate.moving_average(data=sp, win=100, step=1)[:, :-win]  #
        sample_id = neu.sample_id[mask]
        fr_samples = select_trials.get_sp_by_sample(fr, sample_id, samples)
        o1 = np.concatenate((fr_samples["11"], fr_samples["15"]))
        o5 = np.concatenate((fr_samples["51"], fr_samples["55"]))
        c1 = np.concatenate((fr_samples["11"], fr_samples["51"]))
        c5 = np.concatenate((fr_samples["15"], fr_samples["55"]))
        sample = np.concatenate(
            (fr_samples["11"], fr_samples["15"], fr_samples["51"], fr_samples["55"])
        )
        inout_nn_fr.append(sample)
        n0 = fr_samples["0"]
        inout_n0_fr.append(n0)
        # Check selectivity and latency
        color_lat, color_score, color_p = smetrics.get_selectivity(
            c1, c5, win=75, scores=True
        )
        color_selec = (
            np.nan
            if np.isnan(color_lat)
            else "c1" if color_score[color_lat] > 0 else "c5"
        )
        orient_lat, orient_score, orient_p = smetrics.get_selectivity(
            o1, o5, win=75, scores=True
        )
        orient_selec = (
            np.nan
            if np.isnan(orient_lat)
            else "o1" if orient_score[orient_lat] > 0 else "o5"
        )
        neutral_lat, neutral_score, neutral_p = smetrics.get_selectivity(
            sample, n0, win=75, scores=True
        )
        neutral_selec = (
            np.nan
            if np.isnan(neutral_lat)
            else "NN" if neutral_score[neutral_lat] > 0 else "N"
        )

        res["color_lat_" + inout] = color_lat
        res["color_selec_" + inout] = color_selec
        res["color_score_" + inout] = color_score
        res["color_p_" + inout] = color_p
        res["orient_lat_" + inout] = orient_lat
        res["orient_selec_" + inout] = orient_selec
        res["orient_score_" + inout] = orient_score
        res["orient_p_" + inout] = orient_p
        res["neutral_lat_" + inout] = neutral_lat
        res["neutral_selec_" + inout] = neutral_selec
        res["neutral_score_" + inout] = neutral_score
        res["neutral_p_" + inout] = neutral_p
        res["mean_fr_" + inout] = np.nanmean(sp[:, st:] * 1000)

    nnpos_lat, nnpos_score, nnpos_p = smetrics.get_selectivity(
        inout_nn_fr[0], inout_nn_fr[1], win=75, scores=True
    )
    nnpos_selec = (
        np.nan
        if np.isnan(nnpos_lat)
        else "NNin" if nnpos_score[nnpos_lat] > 0 else "NNout"
    )

    neutralpos_lat, neutralpos_score, neutralpos_p = smetrics.get_selectivity(
        inout_n0_fr[0], inout_n0_fr[1], win=75, scores=True
    )
    neutralpos_selec = (
        np.nan
        if np.isnan(neutralpos_lat)
        else "Nin" if neutralpos_score[neutralpos_lat] > 0 else "Nout"
    )

    res["nnpos_lat"] = nnpos_lat
    res["nnpos_selec"] = nnpos_selec
    res["nnpos_score"] = nnpos_score
    res["nnpos_p"] = nnpos_p
    res["neutralpos_lat"] = neutralpos_lat
    res["neutralpos_selec"] = neutralpos_selec
    res["neutralpos_score"] = neutralpos_score
    res["neutralpos_p"] = neutralpos_p
    return res

### Start preprocessing

In [ ]:
area = 'LIP'
rf_loc='in'

In [ ]:
popu = PopulationData.from_python_hdf5(f"/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/population/{area.lower()}/2024_08_28_12_23_36/population.h5")

In [ ]:
df_selectivity = popu.execute_function(
    get_selectivity_info, n_jobs=-1, ret_df=True
)